# BGUDrugChallange2023
### Author: Or Gindes

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

BASE_PATH = ""#'/kaggle/input'
PROJECT_NAME = 'bgudrugchallenge2023'
PROJECT_PATH = os.path.join(BASE_PATH, PROJECT_NAME)

In [2]:
df_train = pd.read_csv(f'{os.path.join(PROJECT_PATH, "train.csv")}')
print(f'shape of training data: {df_train.shape}')
df_train.head()

shape of training data: (332, 4721)


,F_0,F_1,F_2,F_3,F_4,F_5,F_6,F_7,F_8,F_9,...,F_4711,F_4712,F_4713,F_4714,F_4715,F_4716,F_4717,F_4718,F_4719,Y
0,2.948348,-3.501110,-1.295460,-0.473063,3.442854,-0.059270,-7.240580,2.489219,0.238883,0.909147,...,0.094626,0.085609,0.032840,-0.174428,-0.124452,0.141204,0.114121,0.009670,0.052854,0.0
1,0.132907,-0.461278,-2.182900,0.347840,-0.562704,0.459928,-8.376611,0.595829,8.073771,4.659968,...,0.129932,-0.092820,-0.037052,0.065809,-0.075197,-0.042183,0.045762,0.118451,-0.186966,0.0
2,0.456545,0.275523,-1.782677,1.315241,-0.304189,0.188261,-5.266070,0.162359,2.533442,2.168226,...,0.021137,0.034982,0.067085,-0.056314,-0.097726,-0.008313,0.084552,-0.027083,-0.020890,0.0
3,-0.491109,0.530537,-1.700410,2.524352,2.310583,0.252446,-6.055076,-0.587962,8.597134,1.777160,...,0.170188,-0.053689,-0.020882,-0.018316,-0.151553,-0.021109,0.277083,-0.017892,-0.196005,0.0
4,0.429949,0.457843,-0.956712,2.623879,1.084800,1.392040,-5.593976,0.777225,2.017562,-0.161853,...,0.038796,-0.094921,-0.041265,0.044448,0.023078,0.072908,0.030298,-0.045262,-0.083091,0.0


In [3]:
X_test = pd.read_csv(f'{os.path.join(PROJECT_PATH, "validation_and_test.csv")}', index_col='Id')
sample_y = pd.read_csv(f'{os.path.join(PROJECT_PATH, "sample submission.csv")}', index_col="Id")
print(f'shape of test data: {X_test.shape}')
X_test.head()

shape of test data: (143, 4720)


,F_0,F_1,F_2,F_3,F_4,F_5,F_6,F_7,F_8,F_9,...,F_4710,F_4711,F_4712,F_4713,F_4714,F_4715,F_4716,F_4717,F_4718,F_4719
Id,,,,,,,,,,,,,,,,,,,,,
0,-1.428296,0.273314,-0.835889,2.906398,1.221514,-0.212035,-4.865508,0.523301,5.202100,-0.407988,...,0.100275,0.228285,0.019648,0.035425,-0.022730,-0.257698,-0.028127,0.200773,0.088267,-0.202241
1,0.058912,-3.064647,-2.488319,2.099941,-1.163898,-4.883776,-8.703987,3.120910,0.795341,0.483574,...,0.091742,0.066727,0.132647,0.170662,-0.148609,0.029267,0.069381,0.055966,-0.093956,0.072534
2,5.008622,-10.744597,-5.272797,17.425785,-3.502573,-0.205115,-20.726185,-3.344971,12.480215,3.469767,...,-0.116275,0.024817,-0.081762,-0.002624,0.133897,0.075928,0.122732,-0.089868,0.057303,-0.129542
3,0.567597,-3.835310,-3.379487,-0.245614,0.327559,-8.512493,-10.009103,7.537097,0.604305,-0.941639,...,0.082882,0.047993,0.008562,0.032000,-0.099906,0.102638,0.059335,0.006395,-0.073386,0.150540
4,0.196856,-2.251838,-1.782099,5.564932,1.733595,0.940787,-6.924644,1.800831,7.246773,0.430933,...,0.117130,0.082477,-0.002865,0.088535,-0.152767,-0.083517,0.059677,0.092971,0.032295,-0.036221


In [4]:
X_train = df_train.drop('Y', axis=1)
y_train = df_train['Y']

# Stage A: AutoML

This stage will comprise of the following steps:
1. Naive first run with two types of AutoML libraries (AutoSklearn & H2O AutoML)
2. Preprocessing (fill NA, scale data, feature selection)
3. Hyperparameter tuning

### Stage A.1: Naive AutoML (auto-preprocessing)

AutoSklearn

In [15]:
# # Commands to install AutoSklearn in notebook
# !apt-get remove swig
# !apt-get install swig3.0 build-essential -y
# !ln -s /usr/bin/swig3.0 /usr/bin/swig

# !apt-get install build-essential
# !curl https://raw.githubusercontent.com/automl/auto-sklearn/master/requirements.txt | xargs -n 1 -L 1 pip install
# !pip install -U scikit-learn==0.24.0
# !pip install auto-sklearn

In [7]:
from autosklearn.metrics import roc_auc
from autosklearn.classification import AutoSklearnClassifier
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, roc_curve, auc

from smac.optimizer.smbo import SMBO
from smac.runhistory.runhistory import RunInfo, RunValue

# Use an early stop callback when the improvement score is getting low
def callback(
    smbo: SMBO,
    run_info: RunInfo,
    result: RunValue,
    time_left: float
):
    if result.cost <= 0.02:
        return False

# Splitting the data is not necessary because autoML performs cross-validation
automl_1 = AutoSklearnClassifier(seed=42, get_trials_callback=callback, metric=roc_auc)
automl_1.fit(X_train.to_numpy(np.float64), y_train.to_numpy())

y_train_pred = automl_1.predict(X_train)

In [8]:
print("Training Accuracy score:", round(accuracy_score(y_train, y_train_pred), 3))
print("Train set AUC score:", round(roc_auc_score(y_train, automl_1.predict_proba(X_train)[:,1]), 3))

Training Accuracy score: 0.94
Train set AUC score: 0.971


In [10]:
def prep_submission(model, X_test, sub_name='test_submission'):
    submission = pd.DataFrame(model.predict_proba(X_test)[:,1], columns=['Predicted']).reset_index(names="Id")
    submission.to_csv(f"{sub_name}.csv", index=False)

prep_submission(automl_1, X_test, "Naive_AutoSklearn_submission")

Results from ladder board = 0.89065

This model is slightly overfitted (Test AUC is lower than Train AUC). This is likely as a result of the low number of observations left in the training set and the numerous features as well as the default cross-validation method which is kfold which can be an issue if k is too large and each fold will have a small number of observations and be overfitted

In [13]:
print(automl_1.sprint_statistics())

auto-sklearn results:
  Dataset name: ae8a8442-eda5-11ed-b2fa-d0c637f7133b
  Metric: roc_auc
  Best validation score: 0.854353
  Number of target algorithm runs: 272
  Number of successful target algorithm runs: 215
  Number of crashed target algorithm runs: 48
  Number of target algorithms that exceeded the time limit: 6
  Number of target algorithms that exceeded the memory limit: 3



According to the validation score it seems the autoML cross-validation has a good grasp of the actual performance of it's trained models as its much closer to the ladder board score than the training score

In [14]:
automl_1.leaderboard()

,rank,ensemble_weight,type,cost,duration
model_id,,,,,
268,1,0.02,random_forest,0.145647,3.802059
249,2,0.14,random_forest,0.147633,1.497564
226,3,0.04,random_forest,0.149619,2.267734
204,4,0.06,random_forest,0.150281,4.797642
257,5,0.02,random_forest,0.151936,1.595163
177,6,0.02,random_forest,0.152267,4.005137
238,7,0.02,random_forest,0.158226,9.352654
141,8,0.02,random_forest,0.158888,3.049145
175,9,0.04,random_forest,0.158888,3.957441


Another observation is that the best performing models are random_forest models

H2OAutoML


In [26]:
import h2o
from h2o.automl import H2OAutoML
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.18" 2023-01-17; OpenJDK Runtime Environment (build 11.0.18+10-post-Ubuntu-0ubuntu122.04); OpenJDK 64-Bit Server VM (build 11.0.18+10-post-Ubuntu-0ubuntu122.04, mixed mode)
  Starting server from /home/orgindes/anaconda3/envs/MachineLearning_37225214/lib/python3.8/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp183rzuq0
  JVM stdout: /tmp/tmp183rzuq0/h2o_orgindes_started_from_python.out
  JVM stderr: /tmp/tmp183rzuq0/h2o_orgindes_started_from_python.err
  Server is running at http://127.0.0.1:54323
Connecting to H2O server at http://127.0.0.1:54323 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Asia/Jerusalem
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.4
H2O_cluster_version_age:,10 days
H2O_cluster_name:,H2O_from_python_orgindes_jmwels
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.904 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [25]:
train_df = h2o.import_file(os.path.join(PROJECT_PATH, "train.csv"))
y = 'Y'
x = train_df.columns
x.remove(y)
train_df[y] = train_df[y].asfactor()

h2o_automl = H2OAutoML(max_models=50, seed=42, stopping_metric="AUC")
h2o_automl.train(x=x, y=y, training_frame=train_df)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |
19:24:12.350: _train param, Dropping bad and constant columns: [F_1929, F_1373, F_1379, F_1378, F_1499, F_1930, F_1375, F_1374, F_1377, F_1376, F_1497, F_1382, F_1381, F_1384, F_1383, F_1380, F_445, F_1700, F_1942, F_446, F_1941, F_1386, F_1385, F_1470, F_1472, F_1471, F_458, F_1478, F_1599, F_1477, F_1479, F_1474, F_1595, F_455, F_1476, F_1597, F_1475, F_1596, F_1481, F_1480, F_1483, F_460, F_1482, F_472, F_1690, F_1692, F_1691, F_1456, F_1698, F_1455, F_1576, F_1697, F_1458, F_1457, F_1578, F_1699, F_1694, F_1693, F_1454, F_1696, F_1453, F_1695, F_3638, F_1459, F_1461, F_1460, F_1467, F_1466, F_1469, F_1463, F_367, F_1462, F_1465, F_1464, F_1548, F_1549, F_495, F_490, F_491, F_1555, F_1676, F_1554, F_1675, F_1557, F_1678, F_1556, F_1551, F_1550, F_1553, F_1674, F_1438, F_1559, F_1437, F_1558, F_1679, F_1439, F_1560, F_1680, F_380, F_1445, F_1446, F_1441, F_1562, F_1683, F

KeyboardInterrupt: 

In [ ]:
automl_2 = h2o_automl.get_best_model(criterion='auc')

y_train_pred_h20 = automl_2.predict(X_train)

In [ ]:
print("Training Accuracy score:", round(accuracy_score(y_train, y_train_pred_h20), 3))
print("Train set AUC score:", round(roc_auc_score(y_train, automl_2.predict_proba(X_train)[:,1]), 3))

In [ ]:
aml.leaderboard.head(10)

### Stage A.2: AutoML with preprocessing

## Stage A.3: AutoML with hyperparameters (& Preprocessing if it helped)

In [ ]:
# TODO: Add cross validation params

# Use cross-validation with LOOCV or high kfolds? or low kfolds? not sure
automl_1 = AutoSklearnClassifier(
    seed=42, metric="roc_auc",get_trials_callback=callback,
    time_left_for_this_task=3600, per_run_time_limit=300,
    resampling_strategy="holdout",  
    resampling_strategy_arguments={"train_size": 1.0, "shuffle": False},
    preprocessor=["pca", "rfe"]# ,
#     pca__n_components=,  # 10 principal components
#     rfe__n_features_to_select=20,  # 20 features selected by RFE
)
    
automl_1.fit(X_train.to_numpy(np.float), y_train.to_numpy())

y_train_pred = automl_1.predict(X_train)
y_pred = automl_1.predict(X_test)

## Stage A.2: AutoML with my preprocessing

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RepeatedKFold, cross_validate
from sklearn.preprocessing import RobustScaler

Check the distribution of the target parameter and check if the classes are balanced

In [ ]:
y_train.hist(bins=3, )
plt.title('y_train class distribution')
plt.show()

Verify the class are distributed similarly in the test/validation data as well

In [ ]:
sample_y.plot(kind='box')
plt.show()

### Next we'll verify there are is no missing information in the data

In [ ]:
na_cols = X_train.columns[X_train.isna().any()]
na_rows = X_train.isna().any(axis=1)
X_train.loc[na_rows, na_cols]

In [ ]:
# Some missing data was found - 
# since we already have a relatively small amount of data, 
# it is preferable in this case to fill in the missing data 
# instead of dropping the entire row
# NaN values will be replaced with the average value of the columns
X_train = X_train.fillna(X_train.mean())
X_train.loc[na_rows, na_cols]

In [ ]:
# X_test is also filled in with the mean values from X_train
na_cols = X_test.columns[X_test.isna().any()]
na_rows = X_test.isna().any(axis=1)
X_test.loc[na_rows, na_cols]

X_test = X_test.fillna(X_train.mean())
X_test.loc[na_rows, na_cols]

### Scaling - verify all of the features are within the same scale values or scale

In [ ]:
X_train_describe = X_train.describe()
min_range_feature = (X_train_describe.loc['max'] - X_train_describe.loc['min']).idxmin()
max_range_feature = (X_train_describe.loc['max'] - X_train_describe.loc['min']).idxmax()
min_range_feature, max_range_feature

In [ ]:
# Plot feature with max absolute values and 
X_train[[max_range_feature, min_range_feature]].plot(kind='box')
plt.show()

Training a model where some features can take values much larger than others can cause these features to take on more significance for the model, so some scaling is required in this instance to bring all features to the same scale or replace them with categorical features through binning

In [ ]:
# scale data with robustScaler - this is a caler that is robust to larget outliers
scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

### TODO: 
1. When the number of features is much larger than the number of observations, it's usually better to use a regularization method to prevent overfitting (Ridge / Lasso / Elastic Net Regression).

## Stage B: ML Model

In [ ]:
# Models to try: Linear SVC / SVC, KNN, DecisionTree, Ensembles - RandomForest, XgBoost